In [ ]:
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D,\
    Reshape,  multiply, add, Permute

import math
# from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

In [ ]:
# !pip install tensorflow==2.14

In [ ]:

# Load the Drive helper and mount
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import imageio
import numpy as np


def read_images(dir):

    supported_extensions = [".jpg", ".jpeg", ".png"]
    image_list = []
    count = 0
    # Walk through the directory and read images
    for root, _, files in os.walk(dir):
        for file in files:
            file_extension = os.path.splitext(file)[-1].lower()

            # Check if the file is a .jpg or .jpeg image
            if file_extension in supported_extensions:
                image_path = os.path.join(root, file)
                try:
                    image = imageio.imread(image_path)
                    image1 = image

                    image = np.asarray(image)
                    del image1
                    image_list.append(image)
                except Exception as e:
                    print(f"Error reading image {image_path}: {e}")

            count += 1

            if count % 100 == 0:
              print(str(count) + " images read")


    return image_list


def random_rotate(images, rotation_range=(-360, 360)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Rotate(rotate=rotation_range)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images



In [ ]:
dir = os.listdir('drive/MyDrive/testing datasets/apples/test')
print(dir)

['serious_cedar_rust', 'grey_spot', 'serious_scab', 'healthy', 'general_cedar_rust', 'general_scab']


In [ ]:
# Train data directories
base_train_dir = 'drive/MyDrive/testing datasets/apples/test'

general_cedar_rust = read_images(os.path.join(base_train_dir, 'general_cedar_rust'))
general_scab = read_images(os.path.join(base_train_dir, 'general_scab'))
grey_spot = read_images(os.path.join(base_train_dir, 'grey_spot'))
healthy_apple = read_images(os.path.join(base_train_dir, 'healthy'))
serious_cedar_rust = read_images(os.path.join(base_train_dir, 'serious_cedar_rust'))
serious_scab = read_images(os.path.join(base_train_dir, 'serious_scab'))


<ipython-input-4-2ca625451841>:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
200 images read


In [ ]:
labels_healthy_apple = np.zeros(len(healthy_apple))
labels_general_scab = np.ones(len(general_scab))
labels_serious_scab = np.full(len(serious_scab), 2)
labels_grey_spot = np.full(len(grey_spot), 3)
labels_general_cedar_rust = np.full(len(general_cedar_rust),4)
labels_serious_cedar_rust = np.full(len(serious_cedar_rust), 5)


# Combine train labels
labels = np.concatenate([
    labels_healthy_apple,
    labels_general_scab,
    labels_serious_scab,
    labels_grey_spot,
    labels_general_cedar_rust,
    labels_serious_cedar_rust,

])

In [ ]:
# Relative Frequencies
# Create an array to store the frequencies of the lists
class_freq = []

# Append the lengths of the lists to the array
class_freq.append(len(healthy_apple))
class_freq.append(len(general_scab))
class_freq.append(len(serious_scab))
class_freq.append(len(grey_spot))
class_freq.append(len(general_cedar_rust))
class_freq.append(len(serious_cedar_rust))

# Calculate the sum of all frequencies
total_freq = sum(class_freq)

# Calculate the relative frequencies
relative_freq = [freq / total_freq for freq in class_freq]



In [ ]:
images =healthy_apple + general_scab+serious_scab  + grey_spot+ general_cedar_rust  +  serious_cedar_rust


In [ ]:
os.listdir('drive/MyDrive/A2_Models/')

['best_epoch23_densenet121_part2.keras',
 'best_epoch16_densenet121_part1.keras',
 'best_epoch9_densenet121_part1.keras',
 'gamma1_best_epoch21_densenet121_part1.keras',
 'gamma01_best_epoch17_densenet121_part1.keras',
 'batch64_best_epoch17_densenet121_part1.keras',
 'best_epoch42_densenet121expanded_part1.keras',
 'gamma1best_epoch26_densenet121expanded_part3.keras',
 'densenet121expanded_part3.keras',
 'gamma01best_epoch8_densenet121expanded_part2.keras',
 'gamma2best_epoch18_densenet121frozen_part1.keras.keras',
 'gamma2best_epoch18_densenet121frozen_part1.keras',
 'gamma1best_epoch17_densenet121frozen_part3.keras',
 'gamma01best_epoch36_densenet121frozen_part2.keras.keras',
 'crossbest_epoch46_densenet121frozen_part1.keras',
 'gamma2best_epoch22_sedensenet121_part2.keras',
 'gamma1best_epoch20_sedensenet121_part2.keras',
 'gamma01best_epoch20_sedensenet121_part2.keras',
 'crossbest_epoch20_sedensenet121_part2.keras',
 'gamma2best_epoch3_densenet121frozen_part2.keras',
 'crossbest_

In [ ]:
!pip install tensorflow==2.14

In [ ]:
model = keras.models.load_model('epoch366_apple_without_aug_model_part2.keras')

# model = keras.models.load_model('drive/MyDrive/A2_Models/crossbest_epoch24_xception_part1.keras')


In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
import copy

# Find accuracy of model
def find_accuracy(test,pred):
    correct = 0
    total = len(test)

    for i in range(len(test)):
        if test[i] == pred[i]:
            correct += 1

    return correct/total


# Map ANN outputs to classes
def get_labels(y_pred_ann):
    labels = []

    for pred in y_pred_ann:
        max_index = 0

        for i in range(len(pred)):
            if pred[i] > pred[max_index]:
                max_index = i

        labels.append(max_index)

    return labels


def resize_images(images_list, width=128, height=128):
    ia.seed(1)
    # Define the resize augmentation
    resize_augmenter = iaa.Resize({"height": height, "width": width})

    resized_images = []

    for image in images_list:
        # Ensure the image is in RGB format (imgaug uses RGB by default)
        if image.shape[-1] == 1:  # Grayscale image with single channel
            image = np.repeat(image, 3, axis=-1)

        # Apply the resize augmentation
        augmented_image = resize_augmenter.augment_image(image)

        # Append the augmented image to the result list
        resized_images.append(augmented_image)

    return resized_images

def normalize_images(image_list):
  for i in range(len(image_list)):
    image = image_list[i].astype(np.float32) / 255.0
    image_list[i] = image

In [ ]:
images = resize_images(images, 128, 128)
normalize_images(images)
images = np.array(images)

pred = model.predict(images)

16/16 [==============================] - 32s 2s/step


In [ ]:
n_classes = 6

y_pred = get_labels(pred)
total = len(y_pred)

correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == labels[i]:
        correct += 1

accuracy = correct / total

print(correct)
print(total)

# Initialize variables to store weighted sums
weighted_sum_sensitivity = 0
weighted_sum_specificity = 0
weighted_sum_precision = 0
weighted_sum_accuracy = 0
weighted_sum_f1 = 0

class_metrics = []

for i in range(n_classes):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for j in range(len(y_pred)):
        if labels[j] == i and y_pred[j] == i:
            TP += 1
        elif labels[j] != i and y_pred[j] == i:
            FP += 1
        elif labels[j] == i and y_pred[j] != i:
            FN += 1
        elif labels[j] != i and y_pred[j] != i:
            TN += 1


    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else -1
    specificity = TN / (TN + FP) if (TN + FP) > 0 else -1
    precision = TP / (TP + FP) if (TP + FP) > 0 else -1
    class_accuracy = (TP + TN) / (TP + FN + TN + FP)
    recall = sensitivity
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else -1

    class_metrics.append({
        'class': int(i),
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Accuracy': class_accuracy,
        'F1 Score': f1,
        'Relative Frequency': relative_freq[i]
    })

    # Calculate weighted sums
    weighted_sum_sensitivity += sensitivity * relative_freq[i]
    weighted_sum_specificity += specificity * relative_freq[i]
    weighted_sum_precision += precision * relative_freq[i]
    weighted_sum_accuracy += class_accuracy * relative_freq[i]
    weighted_sum_f1 += f1 * relative_freq[i]



print(accuracy)


455
493
0.922920892494929


In [ ]:
# One hot encoding
labels_encoded = []
for label in labels:
    encoding = [0,0,0,0,0,0]
    encoding[int(label)] = 1
    labels_encoded.append(encoding)

labels_encoded = np.array(labels_encoded)


In [ ]:
results = model.evaluate(images, labels_encoded)

16/16 [==============================] - 24s 1s/step - loss: 0.0919 - acc: 0.9229


In [ ]:
import pandas as pd

def create_metrics_dataframe(class_metrics):
    # Create a dictionary to store the metrics for each class
    metrics_dict = {
        'Class': [metrics['class'] for metrics in class_metrics],
        'Specificity': [metrics['Specificity'] for metrics in class_metrics],
        'Sensitivity': [metrics['Sensitivity'] for metrics in class_metrics],
        'Precision': [metrics['Precision'] for metrics in class_metrics],
        'Accuracy': [metrics['Accuracy'] for metrics in class_metrics],
        'F1 Score': [metrics['F1 Score'] for metrics in class_metrics]
    }

    # Create a Pandas DataFrame from the metrics dictionary
    metrics_df = pd.DataFrame(metrics_dict)

    return metrics_df

# Usage:
# Assuming you have class_metrics list defined as in your previous code
metrics_dataframe = create_metrics_dataframe(class_metrics)

# Calculate weighted average metrics
weighted_avg_metrics = {
    'Class': 'Weighted Avg',
    'Specificity': weighted_sum_specificity,  # Replace with your actual values
    'Sensitivity': weighted_sum_sensitivity,  # Replace with your actual values
    'Precision': weighted_sum_precision,      # Replace with your actual values
    'Accuracy': weighted_sum_accuracy,        # Replace with your actual values
    'F1 Score': weighted_sum_f1               # Replace with your actual values
}


# Append the weighted average metrics to the dataframe
metrics_dataframe = metrics_dataframe.append(weighted_avg_metrics, ignore_index=True)

metrics_dataframe = metrics_dataframe.round(3)

print(metrics_dataframe)

metrics_dataframe.to_csv('classwise_metrics.csv', index=False)


          Class  Specificity  Sensitivity  Precision  Accuracy  F1 Score
0             0        0.977        0.967      0.981     0.972     0.974
1             1        0.962        0.792      0.691     0.945     0.738
2             2        0.987        0.647      0.786     0.963     0.710
3             3        0.992        0.990      0.970     0.992     0.980
4             4        0.989        0.939      0.861     0.986     0.899
5             5        0.996        0.556      0.714     0.988     0.625
6  Weighted Avg        0.981        0.923      0.924     0.974     0.922


<ipython-input-18-62d661c5ca5e>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_dataframe = metrics_dataframe.append(weighted_avg_metrics, ignore_index=True)


In [ ]:
for i in range(len(y_pred)):
  print(y_pred[i], labels[i])

0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
3 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
2 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.

In [ ]:
print(accuracy)

0.922920892494929


In [ ]:
from mlxtend.data import loadlocal_mnist

In [ ]:
model.optimizer.learning_rate

<tf.Variable 'current_learning_rate:0' shape=() dtype=float32, numpy=1e-04>